# Setup Cromwell Input

Starting a job on `cromwell` requires a source wdl and inputs to be configured. This notebook helps configure inputs and submits the job.

In [ ]:
import json
from ipywidgets import widgets

In [ ]:
!wget https://github.com/broadinstitute/cromwell/releases/download/84/womtool-84.jar -O womtool.jar

## Setup variables

In [ ]:
MAIN_WORKFLOW = "GvsJointVariantCalling"
WDL_FILE = f"{MAIN_WORKFLOW}.wdl"

In [ ]:
import os

CROMWELL_EXAMPLES_DIR=os.path.expanduser('~/terra-tutorials/cromwell')
GATK_DIR=f'{CROMWELL_EXAMPLES_DIR}/gatk'
CROMWELL_SERVER_LOG=f'{CROMWELL_EXAMPLES_DIR}/cromwell.server.log'

!mkdir -p {CROMWELL_EXAMPLES_DIR}

print(f'gatk git repo will be cloned to {GATK_DIR}')


## Build zipped wdls

In [ ]:
%%bash -s {GATK_DIR}

GATK_DIR="$1"

git clone https://github.com/broadinstitute/gatk.git "${GATK_DIR}"
cd "${GATK_DIR}"
git checkout ah_var_store

In [ ]:
%%bash -s {GATK_DIR}

GATK_DIR="$1"

mkdir -p gvs_wdls
# We need all the gvs wdls...
cp "${GATK_DIR}"/scripts/variantstore/wdl/*wdl gvs_wdls
# ...and these two gvs wdls
cp "${GATK_DIR}"/scripts/vcf_site_level_filtering_wdl/JointVcfFiltering.wdl gvs_wdls
cp "${GATK_DIR}"/scripts/variantstore/variant_annotations_table/GvsCreateVATFilesFromBigQuery.wdl gvs_wdls

# For now, we can't do relative imports because we're zipping all the wdls together
sed -i 's|import "../../vcf_site_level_filtering_wdl/JointVcfFiltering.wdl" as VQSRLite|import "JointVcfFiltering.wdl" as VQSRLite|g' gvs_wdls/GvsCreateFilterSet.wdl
sed -i 's|import "../variant_annotations_table/GvsCreateVATFilesFromBigQuery.wdl" as GvsCreateVATFilesFromBigQuery|import "GvsCreateVATFilesFromBigQuery.wdl" as GvsCreateVATFilesFromBigQuery|g' gvs_wdls/GvsCreateVATfromVDS.wdl

# We also need the "main" wdl
cp gvs_wdls/GvsJointVariantCalling.wdl .

# Delete existing zip if exists
rm -f gvs_wdls.zip
# Create zip file of wdls. -j so we don't include folders.
zip -rj gvs_wdls.zip gvs_wdls

## Parse out inputs

In [ ]:
required_input_list = []
for input_line in inputs_json:
    if 'optional' in input_line or '{' in input_line or '}' in input_line or not input_line:
        continue
    req_input = input_line.split(': ')[0].strip().replace('"', '')
    required_input_list.append(req_input.strip())
print(required_input_list)

In [ ]:
def handle_submit(change):
    # Used for debugging
    #print(change)
    pass

display_elements = []
input_text_boxes = []
for input_key in required_input_list:
    text = widgets.Text()
    text.observe(handle_submit, names='value')
    input_text_boxes.append(text)
    hbox = widgets.HBox([widgets.Label(f'{input_key}:'), text])
    display_elements.append(hbox)
    
box = widgets.VBox(display_elements)
display(box)

    

## Build json input file

In [ ]:
input_text_box_values = [box.value for box in input_text_boxes]
input_as_dict = dict(zip(required_input_list, input_text_box_values))
input_as_dict

In [ ]:
with open('gvs.inputs', 'w') as outfile:
    json.dump(input_as_dict, outfile, indent=4)

## Submit job to server

In [ ]:
import requests

url = "http://localhost:8000/api/workflows/v1"

files = {
    'workflowSource': ('file', open(WDL_FILE, 'rb')),
    'workflowDependencies': ('file', open('gvs_wdls.zip', 'rb')),
    'workflowInputs': ('file', open('gvs.inputs', 'rb'))
}

headers = {
    'Accept': 'application/json'
}

response = requests.post(url, headers=headers, files=files)
response.content

In [ ]:
%%bash -s {WDL_FILE}

# To run a shell command in the background from an iPython notebook, we need to use the
# %%bash magic instead of the "!" notation.

WDL_FILE="$1"
curl -X POST --header "Accept: application/json"\
    -v "localhost:8000/api/workflows/v1" \
    -F workflowSource=@"${WDL_FILE}" \
    -F workflowDependencies=@cellranger_wdls.zip \
    -F workflowInputs=@cellranger.inputs

### Check status of job

In [ ]:
!curl http://localhost:8000/api/workflows/v1/runs/d689a1d1-38c2-48ea-a454-ebdf02b62a7b/status

In [ ]:
!tail -n 15 {CROMWELL_SERVER_LOG}

# Observe Cromwell output